# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [1]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [2]:
artists = {'dualipa':"https://www.azlyrics.com/d/dualipa.html",
           'hozier':"https://www.azlyrics.com/h/hozier.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: Based on the robots.txt file, web scraping in generally allowed. Here is the results: 

* User-agent: *
* Disallow: /lyricsdb/
* Disallow: /song/
* Allow: /

* User-agent: 008
* Disallow: /

The directories /lyricsdb/ and /song/ are not allowed, but all the other directories are ok. And since we're going to be scraping the /lyrics/ directory, we are good to go.


In [3]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page) # gets webpage content of the url that was provided earlier
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, 'html.parser') #creates a beautiful soup object to access the html content
    links = soup.find_all('a', href=True)
    lyric_links = [link['href'] for link in links if '/lyrics/' + artist.lower() in link['href']]
    base_url = 'https://www.azlyrics.com/'
    lyric_links = [base_url + link.lstrip('..') for link in lyric_links]
    
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    lyrics_pages[artist] = lyric_links
    

In [4]:
lyrics_pages

defaultdict(list,
            {'dualipa': ['https://www.azlyrics.com//lyrics/dualipa/genesis.html',
              'https://www.azlyrics.com//lyrics/dualipa/lastdance.html',
              'https://www.azlyrics.com//lyrics/dualipa/hotterthanhell.html',
              'https://www.azlyrics.com//lyrics/dualipa/betheone.html',
              'https://www.azlyrics.com//lyrics/dualipa/idgaf.html',
              'https://www.azlyrics.com//lyrics/dualipa/blowyourmindmwah.html',
              'https://www.azlyrics.com//lyrics/dualipa/newlove.html',
              'https://www.azlyrics.com//lyrics/dualipa/nogoodbyes.html',
              'https://www.azlyrics.com//lyrics/dualipa/thinkingboutyou.html',
              'https://www.azlyrics.com//lyrics/dualipa/roomfor2.html',
              'https://www.azlyrics.com//lyrics/dualipa/lostinyourlight.html',
              'https://www.azlyrics.com//lyrics/dualipa/badtogether.html',
              'https://www.azlyrics.com//lyrics/dualipa/garden.html',
        

Here is the link to the html code for the artist Hozier - view-source:https://www.azlyrics.com/h/hozier.html

* The "links" variable stores looks at the html code for the webpage where it has <'a'> tags, indicating hyperlinks. 
* The "lyric_links" variable further filters the "links" variable to only show the hyperlinks that are lyrics of the artist.
* The most recent "lyrics_links" puts the base_url and the previous link together to create a working url.

Let's make sure we have enough lyrics pages to scrape. 

In [5]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [6]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For dualipa we have 112.
The full pull will take for this artist will take 0.31 hours.
For hozier we have 71.
The full pull will take for this artist will take 0.2 hours.


There's actually 124 songs for dua lip, but the code wasn't able to pull some of them since the format was different for some songs. Instead of 
* <'div' class="listalbum-item"><'a' href="/lyrics/dualipa/prettyplease.html" target="_blank">Pretty Please</'a'></'div'>  
it was

* <'div' class="listalbum-item"><'a' href="https://www.azlyrics.com/lyrics/mileycyrus/prisoner.html" target="_blank">Prisoner</'a'><'div' class="comment">(The Moonlight Edition Bonus Track)</'div'></'div'>

There were 2 formats that were used when creating this html page. For this assignment however, i think 112 songs is suffice.

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [7]:
def generate_filename_from_link(link) :
    
    if not link :
        return None

    # drop the http or https and the html
    name = link
    name = name.replace("https://","").replace("http://","")
    name = name.replace(".html","")

    name = name.replace("/lyrics/","")

    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.
# checking if the directory is correct
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\jessh\Documents\ADS509


### 1. Build a subfolder for the artist

In [9]:
# making a new folder called lyrics
os.mkdir("lyrics")

#if os.path.isdir("lyrics") : 
#    shutil.rmtree("lyrics/")

#os.mkdir("lyrics")

In [10]:
# changing directory
lyrics_directory = "C:/Users/jessh/Documents/ADS509/lyrics"
os.chdir(lyrics_directory)
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: C:\Users\jessh\Documents\ADS509\lyrics


In [11]:
# Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist

start = time.time()

# iterate over the artist and their lyrics pages
for artist in lyrics_pages :

    # Use this space to carry out the following steps: 
    # creating a folder for the artist the loop is currently on
    artist_dir = os.mkdir(artist)
    
    # changing directory to the artist folder
    new_directory = "C:/Users/jessh/Documents/ADS509/lyrics/" + artist
    os.chdir(new_directory)
    
    # Limit the number of links to 20
    limited_pages = lyrics_pages[artist][:20]
    
    for page in limited_pages:
        
        # getting the filename from link
        name = generate_filename_from_link(page)
        
        #3. Request the lyrics page. 
        # sending a GET request to the link
        response = requests.get(page)
        time.sleep(5 + 10*random.random())
        
        # checking if the request was successful
        if response.status_code == 200:
            
            # 4. Extract the title and lyrics from the page.
            # parsing the HTML content
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # finding the div containing the lyrics
            lyrics_div = soup.find('div', class_='col-xs-12 col-lg-8 text-center')
            
            #5. Write out the title, two returns ('\n'), and the lyrics.
            # getting the lyrics
            lyrics = lyrics_div.text.strip()
                
            # finding the index of "Submit Corrections"
            submit_index = lyrics.find("Submit Corrections")

            # only getting the lyrics portion, the "Submit Corrections" portion was after the lyrics portion
            lyrics = lyrics[:submit_index]

            # Save the lyrics to a file
            with open(f"{name}", "w", encoding="utf-8") as file:
                    file.write(lyrics)
                        
    os.chdir(lyrics_directory)
    

In [12]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.12 hours.


In [13]:
orig_directory = "C:/Users/jessh/Documents/ADS509"
os.chdir(orig_directory)

---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [14]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [15]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

In [16]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For dualipa we have 20 files.
For dualipa we have roughly 7494 words, 718 are unique.
For hozier we have 20 files.
For hozier we have roughly 5710 words, 1021 are unique.
